In [81]:
import json
from snowflake.snowpark.session import Session
import snowflake.snowpark.functions as F

connection_parameters = json.load(open('../connection.json'))
session = Session.builder.configs(connection_parameters).create()

### 1. Build docker image and push the image to image registry

- Run the below code from terminal or execute the next cell by updating the connection details

<!-- Run below commands from a terminal. Update the ORGNAME-ACCTNAME and username -->
``` bash
cd ..
cd text2sql
```
> be sure your directory has changed to text2sql (validate via pwd)
``` bash
docker build --no-cache --platform linux/amd64 -t ORGNAME-ACCTNAME.registry.snowflakecomputing.com/llmdemo/public/images/audiollm:latest . 

docker login ORGNAME-ACCTNAME.registry.snowflakecomputing.com -u <username>

docker push ORGNAME-ACCTNAME.registry.snowflakecomputing.com/llmdemo/public/images/audiollm:latest
```

###  2. Creating Text2SQL SPC Service

Update the image line in [llm-text2sql.yaml](./llm-text2sql.yaml) to provide your orgname-accountname values

ex. image: <i>myorg-myaccount</i>.registry.snowflakecomputing.com/pr_llmdemo/public/image_repo/whisper-audio2text:latest

PS: <b>Run following commands using the SPCS Role, not accountadmin! (this should already be set in your [connection.json](../connection.json) file) </b>


In [82]:

session.file.put("./llm-text2sql.yaml", "@specs",auto_compress=False)

[PutResult(source='llm-text2sql.yaml', target='llm-text2sql.yaml', source_size=648, target_size=648, source_compression='NONE', target_compression='NONE', status='UPLOADED', message='')]

In [83]:
# Create the required stage
stages=['LLM_WORKSPACE']
for stg in stages:
    session.sql(f'''
                CREATE STAGE IF NOT EXISTS {stg} ENCRYPTION = (TYPE = 'SNOWFLAKE_SSE')
                DIRECTORY = (ENABLE = TRUE);
                ''').collect()

In [85]:
# Create the service. EXTERNAL_ACCESS_INTEGRATIONS(ALLOW_ALL_EAI) is created in audio2text/audio2text_setup_code.ipynb
session.sql('''
create service if not exists llama_text2sql_svc
in compute pool PR_GPU_7
from @specs
SPECIFICATION_FILE='llm-text2sql.yaml'
EXTERNAL_ACCESS_INTEGRATIONS = (ALLOW_ALL_EAI)
            ''').collect()

[Row(status='Service LLAMA_TEXT2SQL_SVC successfully created.')]

> The service must be in Ready State to proceed. Run the following command to confirm before proceeding to next step.

In [89]:
#  Check the status of service
import ast
res=session.sql(''' 
SELECT SYSTEM$GET_SERVICE_STATUS('llama_text2sql_svc',1)
''').collect()[0][0]
ast.literal_eval(res)[0]

{'status': 'READY',
 'message': 'Running',
 'containerName': 'text2sql-container',
 'instanceId': '0',
 'serviceName': 'LLAMA_TEXT2SQL_SVC',
 'image': 'sfsenorthamerica-demo-psheehan.registry.snowflakecomputing.com/llmdemo/public/images/audiollm:latest',
 'restartCount': 0,
 'startTime': '2024-06-21T18:09:42Z'}

>  Be sure the service is in the Ready State before procedding.

### 4. Accessing Jupyter Lab Endpoints
Wait for the status of the service to be Running before moving to next step. 

Run the below query to get the api endpoint for the <b> jupyter lab </b>. Get the <b>ingress_url </b>(this endpoint will launch jupyter notebook running Snowpark Containers)  from the below query output.

Ouput that you would get after running the below query.

Row(name='llm-audio-ep', port=8888, protocol='TCP', ingress_enabled='true', ingress_url='test123-us-west-ccarrero-452.snowflakecomputing.app')

In [90]:
session.sql('''show endpoints in service llama_text2sql_svc;
            ''').collect()[0]

# Row(name='llm-audio-ep', port=8888, protocol='TCP', ingress_enabled='true', ingress_url='test123-us-west-ccarrero-452.snowflakecomputing.app')

Row(name='api', port=9000, port_range=None, protocol='HTTP', is_public='true', ingress_url='yjbzi-sfsenorthamerica-demo-psheehan.snowflakecomputing.app')

### 5. Perform the following action on the Jupyter lab which is accessible from the endpoint

<b> 1. Open the terminal in the JupyterLab(which is running in Snowpark Containers) and run the below shell script  </b>

- sh [./download_model.sh](https://github.com/Snowflake-Labs/sfguide-call-centre-analytics-with-snowflake-cortex-and-spcs/blob/main/text2sql/download_model.sh)

This file will download the Numberstation model from hugging face and downloads to the stage which is mounted

<b> 2. Run the [FineTuneModel.ipynb](https://github.com/Snowflake-Labs/sfguide-call-centre-analytics-with-snowflake-cortex-and-spcs/blob/main/text2sql/FineTuneModel.ipynb)  </b>

* Execute each cell by cell where we are fine tuning the model as well with our own dataset.

* After executing the last cell you will be running a fast api service which will expose the Number station LLM as a api endpoint.

><b> PS </b> : You need to execute step 1 and 2 only one time and from next time after you start the Snowpark service, run the below command from the terminal launched in the jupyter lab:

 * <b> sh [RunFastAPI.sh](https://github.com/Snowflake-Labs/sfguide-call-centre-analytics-with-snowflake-cortex-and-spcs/blob/main/text2sql/RunFastAPI.sh) </b>



### 6. Execute the below commands to create the service function

In [ ]:
session.sql('''CREATE OR REPLACE FUNCTION text2sql(text TEXT)
RETURNS VARIANT
SERVICE=llama_text2sql_svc
ENDPOINT=api
AS '/text2sql
            ''').collect()


In [ ]:
#Testing the function
session.sql('''
select text2sql('What is the distinct purpose of the calls in the last month?')::string;
''').collect()